## Setup

First, we need to install the necessary library and set up the Groq API client.

Make sure you have your Groq API key stored in Colab secrets under the name `GROQ_API_KEY`.

In [ ]:
!pip install groq ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.1 MB/s eta 0:00:00


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from groq import Groq
import os
from google.colab import userdata, files

# Initialize Groq client using Colab secrets
api_key = userdata.get('GROQ_API_KEY')
client = Groq(api_key=api_key)
print("API key successfully loaded from Colab secrets")

API key successfully loaded from Colab secrets


## Upload Articles

Now, upload the three text files containing your research articles.

In [ ]:
# Upload files widget
print("Please upload exactly 3 text files containing research articles:")
uploaded = files.upload()

# Store file contents and filenames
file_contents = []
filenames = []

if len(uploaded) != 3:
    print("Please upload exactly 3 files")
else:
    for filename, content in uploaded.items():
        article_text = content.decode('utf-8')
        file_contents.append(article_text)
        filenames.append(filename)
        print(f"Loaded: {filename} ({len(article_text.split())} words)")

# A simple validation function (no length restriction)
def validate_article_length(article, filename):
    # No length restriction for this example
    return None

Please upload exactly 3 text files containing research articles:


Saving ChatGPT Search A Guide With Example.txt to ChatGPT Search A Guide With Example.txt
Saving Context Engineering A Guide With Ex.txt to Context Engineering A Guide With Ex.txt
Saving Small Language Models A Guide With.txt to Small Language Models A Guide With.txt
Loaded: ChatGPT Search A Guide With Example.txt (2203 words)
Loaded: Context Engineering A Guide With Ex.txt (2085 words)
Loaded: Small Language Models A Guide With.txt (2457 words)


## Define Prompting Functions

We'll define functions for zero-shot, one-shot, and few-shot prompting based on the desired summary format.

In [ ]:
# Zero-shot prompt with explicit headings
def zero_shot_prompt(article):
    return f"""
    Summarize the following research article in a single paragraph of 150 words or less. Include the following sections:
    Background/introduction: [Content]
    Research objective: [Content]
    Methodology: [Content]
    Key results: [Content]
    Main takeaways/conclusions: [Content]

    Article:
    {article}
    """

# One-shot prompt with explicit headings (uses first article's summary as example)
def one_shot_prompt(article, example_article, example_summary):
    return f"""
    Summarize the following research article in a single paragraph of 150 words or less. Include the following sections:
    Background/introduction: [Content]
    Research objective: [Content]
    Methodology: [Content]
    Key results: [Content]
    Main takeaways/conclusions: [Content]

    Example Article:
    {example_article}

    Example Summary:
    {example_summary}

    Now summarize this article, including the explicit headings for each section:
    {article}
    """

# Few-shot prompt with explicit headings (uses first two articles and their summaries as examples)
def few_shot_prompt(article, examples):
    prompt = """
    Summarize the following research article in a single paragraph of 150 words or less. Include the following sections:
    Background/introduction: [Content]
    Research objective: [Content]
    Methodology: [Content]
    Key results: [Content]
    Main takeaways/conclusions: [Content]

    """

    for i, (ex_article, ex_summary) in enumerate(examples):
        prompt += f"\nExample {i+1}:\nArticle: {ex_article}\nSummary: {ex_summary}\n"

    prompt += f"\nNow summarize this article, including the explicit headings for each section:\n{article}"
    return prompt

## Define API Call Function

This function sends the prompt to the Groq API and returns the generated summary. We will use the `meta-llama/llama-4-scout-17b-16e-instruct` model as requested.

In [ ]:
# Function to call Groq API
def get_summary(prompt):
    completion = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct", # Using the requested model
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_completion_tokens=200, # Keep max tokens for summary consistent
        top_p=1,
        stream=False,
        stop=None,
    )
    return completion.choices[0].message.content.strip()

# Function to validate word count of the summary
def validate_summary(summary):
    word_count = len(summary.split())
    if word_count > 150:
        return f"Word count: {word_count}"
    else:
        return f"Word count: {word_count}"

## Process Articles Sequentially

This section sets up the UI and the logic to process the uploaded articles sequentially using the defined prompting techniques and save the output.

In [ ]:
# Create UI widgets
process_button = widgets.Button(
    description='Process Articles Sequentially and Save Output',
    button_style='success',
    tooltip='Process all three articles using sequential prompting techniques and save output to a file'
)

output_area = widgets.Output()

# Display UI
display(widgets.VBox([process_button, output_area]))

# Button click handler for sequential processing and saving
def on_process_clicked(b):
    if len(file_contents) != 3:
        with output_area:
            clear_output()
            print("Please upload exactly 3 files first")
        return

    # Validate all articles first (no length validation now)
    validation_errors = []
    for i, (article, filename) in enumerate(zip(file_contents, filenames)):
        error = validate_article_length(article, filename)
        if error:
            validation_errors.append(error)

    if validation_errors:
        with output_area:
            clear_output()
            print("Article validation errors:")
            for error in validation_errors:
                print(error)
        return

    output_text = ""
    with output_area:
        clear_output()
        print("Starting sequential processing of articles...\n")

        # Store summaries for use in subsequent prompts
        summaries = []

        # Process Article 1 with Zero-shot
        output_text += "="*60 + "\n"
        output_text += f"Processing Article 1: {filenames[0]}\n"
        output_text += "="*60 + "\n"
        print("="*60)
        print(f"Processing Article 1: {filenames[0]}")
        print("="*60)
        zero_shot = zero_shot_prompt(file_contents[0])
        summary1 = get_summary(zero_shot)
        summaries.append(summary1)

        output_text += "Zero-shot Summary:\n"
        output_text += summary1 + "\n"
        output_text += validate_summary(summary1) + "\n\n"
        print("Zero-shot Summary:")
        print(summary1)
        print(validate_summary(summary1))
        print("\n")

        # Process Article 2 with One-shot (using Article 1's summary as example)
        output_text += "="*60 + "\n"
        output_text += f"Processing Article 2: {filenames[1]}\n"
        output_text += "="*60 + "\n"
        print("="*60)
        print(f"Processing Article 2: {filenames[1]}")
        print("="*60)
        one_shot = one_shot_prompt(file_contents[1], file_contents[0], summary1)
        summary2 = get_summary(one_shot)
        summaries.append(summary2)

        output_text += "One-shot Summary:\n" # Simplified heading
        output_text += summary2 + "\n"
        output_text += validate_summary(summary2) + "\n\n"
        print("One-shot Summary:") # Simplified heading
        print(summary2)
        print(validate_summary(summary2))
        print("\n")


        # Process Article 3 with Few-shot (using Articles 1&2 and their summaries)
        output_text += "="*60 + "\n"
        output_text += f"Processing Article 3: {filenames[2]}\n"
        output_text += "="*60 + "\n"
        print("="*60)
        print(f"Processing Article 3: {filenames[2]}")
        print("="*60)
        examples = [(file_contents[0], summary1), (file_contents[1], summary2)]
        few_shot = few_shot_prompt(file_contents[2], examples)
        summary3 = get_summary(few_shot)
        summaries.append(summary3)

        output_text += "Few-shot Summary:\n" # Simplified heading
        output_text += summary3 + "\n"
        output_text += validate_summary(summary3) + "\n\n"
        print("Few-shot Summary:") # Simplified heading
        print(summary3)
        print(validate_summary(summary3))
        print("\n")


        # Final summary of all approaches
        output_text += "="*60 + "\n"
        output_text += "FINAL SUMMARY OF ALL APPROACHES\n"
        output_text += "="*60 + "\n"
        print("="*60)
        print("FINAL SUMMARY OF ALL APPROACHES")
        print("="*60)

        output_text += f"Article 1 ({filenames[0]}): Zero-shot approach\n"
        output_text += f"Word count: {len(file_contents[0].split())}\n"
        output_text += f"Summary word count: {len(summary1.split())}\n\n"
        print(f"Article 1 ({filenames[0]}): Zero-shot approach")
        print(f"Word count: {len(file_contents[0].split())}")
        print(f"Summary word count: {len(summary1.split())}")
        print("\n" + "-"*40 + "\n")

        output_text += f"Article 2 ({filenames[1]}): One-shot approach (with Article 1 example)\n"
        output_text += f"Word count: {len(file_contents[1].split())}\n"
        output_text += f"Summary word count: {len(summary2.split())}\n\n"
        print(f"Article 2 ({filenames[1]}): One-shot approach (with Article 1 example)")
        print(f"Word count: {len(file_contents[1].split())}")
        print(f"Summary word count: {len(summary2.split())}")
        print("\n" + "-"*40 + "\n")

        output_text += f"Article 3 ({filenames[2]}): Few-shot approach (with Articles 1&2 examples)\n"
        output_text += f"Word count: {len(file_contents[2].split())}\n"
        output_text += f"Summary word count: {len(summary3.split())}\n"
        output_text += "\n" + "="*60
        print(f"Article 3 ({filenames[2]}): Few-shot approach (with Articles 1&2 examples)")
        print(f"Word count: {len(file_contents[2].split())}")
        print(f"Summary word count: {len(summary3.split())}")
        print("\n" + "="*60)

    # Save output to a text file
    output_filename = "sequential_summaries.txt"
    with open(output_filename, "w") as f:
        f.write(output_text)

    with output_area:
        print(f"\nOutput saved to '{output_filename}'")


# Attach event handler
process_button.on_click(on_process_clicked)